In [4]:
# importations
import sys
sys.path.append('notebooks')

import numpy as np
import pandas as pd
from pyedflib import EdfReader
import mne
import EntropyHub

import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.colors import Normalize
from scipy.interpolate import PchipInterpolator,interp1d

import joblib

%matplotlib inline

import seaborn as sns

import copy
import os
import importlib

import generisAPI
importlib.reload(generisAPI)

import generisAPI.file_operators as fp
importlib.reload(generisAPI.file_operators)

import generisAPI.data_cleaning as dcln
importlib.reload(generisAPI.data_cleaning)

import generisAPI.preprocessing as ppcn
importlib.reload(generisAPI.preprocessing)

import generisAPI.processing as pcn
importlib.reload(generisAPI.processing)

import generisAPI.audProcessing as audpcn
importlib.reload(generisAPI.audProcessing)

import generisAPI.plots as plots
importlib.reload(generisAPI.plots)

import generisAPI.fuzzEntropy as fuzzEnt
importlib.reload(generisAPI.fuzzEntropy)

import generisAPI.featuresExtraction as features
importlib.reload(generisAPI.featuresExtraction)

<module 'generisAPI.featuresExtraction' from 'notebooks\\generisAPI\\featuresExtraction.py'>

In [5]:
# definitions
electrodes=[
    'Fp1','Fp2','F3','F4','C3',
    'C4','P3','P4','O1','O2',
    'F7','F8','T3','T4','T5',
    'T6','Fz','Pz','Cz'
    ]
len(electrodes)

19

In [6]:
# compile eeg_data and annotations
subjects_dataset_root_path = 'Acquired_Dataset/'

data = fp.xtract_id_subject_info_dict(subjects_dataset_root_path)
print(len(data))

32


In [7]:
# assert eeg_data trials match marker trials

trialMarkerLength = dcln.assertTrialsMarkersLength()
trialMarkerLength.fit(data)
print(trialMarkerLength.indices)
clean_data = trialMarkerLength.transform(data,False)

[]


In [8]:
# check for null eeg_data

null_data_checker = dcln.nullIndicesTracer()
null_data_checker.fit(clean_data)
null_data_checker.indices['1']={}
null_data_checker.indices['2']={}
print(null_data_checker.indices)
clean_data = null_data_checker.transform(clean_data,True)

{'1': {}, '16': {'rest1': [0, 1], 'arith': [0, 1], 'rest2': [0, 1], 'auditory': [0, 1]}, '2': {}, '20': {'rest1': [], 'arith': [], 'rest2': [], 'auditory': [0]}, '23': {'rest1': [], 'arith': [0], 'rest2': [0], 'auditory': [0]}, '3': {'rest1': [0, 1], 'arith': [0, 1], 'rest2': [0, 1], 'auditory': [0, 1]}, '32': {'rest1': [0, 1, 2], 'arith': [0, 1, 2], 'rest2': [0, 1, 2], 'auditory': [0, 1, 2]}, '4': {'rest1': [0, 1], 'arith': [0, 1], 'rest2': [0, 1, 2], 'auditory': [0, 1, 2]}, '5': {'rest1': [0, 1], 'arith': [0, 1], 'rest2': [0, 1], 'auditory': [0, 1]}, '6': {'rest1': [], 'arith': [], 'rest2': [], 'auditory': [0]}, '8': {'rest1': [0, 1], 'arith': [0, 1], 'rest2': [0, 1], 'auditory': [0, 1]}, '9': {'rest1': [0, 1], 'arith': [0, 1], 'rest2': [0, 1], 'auditory': [0, 1]}}


In [9]:
#prerprocessing
preprocess_method=[
    [ppcn.channel_dropper,{'index':[19,20,21,22,23],'axis':0}],
    [ppcn.filter,{'lfreq':75,'hfreq':45,'sfreq':200}],
    [ppcn.filter,{'lfreq':100,'hfreq':None,'sfreq':200}],
    # [ppcn.resampler,{'up':1,'down':1}],
    [ppcn.baseline_corrector,{'with_std':False}]
]

preprocess_pipeline = ppcn.ppcn_pipeline(preprocess_method)

mne.set_log_level('ERROR')
clean_data = ppcn.all_subjects_preprocessor(
    preprocess_pipeline,
    clean_data
)